In [46]:
with open("input.txt") as f:
    lines = f.readlines()
    lines = list(map(lambda x: x.strip(), lines))

In [47]:
class Sensor:
    def __init__(self, x,y,distance):
        self.x = x
        self.y = y
        self.distance = distance

In [48]:
import numpy as np
def getManhattanDistance(sensor, beacon):
    return abs(sensor[0]-beacon[0])+abs(sensor[1]-beacon[1])

sensors = []
beacons = {}
mapXMin = 0
mapXMax = 0
mapYMin = 0
mapYMax = 0

for line in lines:
    parts = line.split(" ")
    sensorX = int(parts[2].split("=")[-1][:-1])
    sensorY = int(parts[3].split("=")[-1][:-1])
    beaconX = int(parts[-2].split("=")[-1][:-1])
    beaconY = int(parts[-1].split("=")[-1])

    distance = getManhattanDistance((sensorX, sensorY), (beaconX, beaconY))
    if sensorX-distance < mapXMin: mapXMin = sensorX-distance
    if beaconX-distance < mapXMin: mapXMin = beaconX-distance
    if sensorX+distance > mapXMax: mapXMax = sensorX+distance
    if beaconX+distance > mapXMax: mapXMax = beaconX+distance

    if sensorY-distance < mapYMin: mapYMin = sensorY-distance
    if beaconY-distance < mapYMin: mapYMin = beaconY-distance
    if sensorY+distance > mapYMax: mapYMax = sensorY+distance
    if beaconY+distance > mapYMax: mapYMax = beaconY+distance

    sensor = Sensor(sensorX,sensorY, distance)
    sensors+=[sensor]
    beacons[(beaconX, beaconY)]=(beaconX, beaconY)

def pointsCoveredBySensorInRow(sensor, y):
    distanceToLine = abs(sensor.y - y)
    if distanceToLine > sensor.distance:
        return set({})
    else:
        rest = sensor.distance-distanceToLine
        xMin = sensor.x-rest
        xMax = sensor.x+rest
        return set(range(xMin, xMax+1))

yIndex = 2000000
listOfCoveredPoints = list(map(lambda sensor: pointsCoveredBySensorInRow(sensor, yIndex), sensors))
beaconsInRow = sum(list(map(lambda beacon: beacon[1] == yIndex, beacons)))
coveredPoints = set({})
for pointSet in listOfCoveredPoints:
    coveredPoints = coveredPoints.union(pointSet)

len(coveredPoints)-beaconsInRow

4737567

In [49]:
from functools import reduce

def getPossiblePoints(sensor, startPoint, endPoint, limit, shift):
    xMovingDirection = int((endPoint[0]-startPoint[0])/(abs(endPoint[0]-startPoint[0])))
    yMovingDirection = int((endPoint[1]-startPoint[1])/(abs(endPoint[1]-startPoint[1])))

    points = set({})
    for step in range(0, sensor.distance+1):
        borderPointX = startPoint[0]+ xMovingDirection*step
        borderPointY = startPoint[1]+ yMovingDirection*step

        newX = borderPointX+shift
        if newX >= 0 and newX <= limit and borderPointY>=0 and borderPointY<=limit:
            points.add((newX, borderPointY))
    return points

def getPointsNextBorderOfSensor(sensor, limit):
    print("SENSOR CHECK1: ", sensor.x, sensor.y, sensor.distance)
    sensorXMax = (sensor.x+sensor.distance, sensor.y)
    sensorXMin = (sensor.x-sensor.distance, sensor.y)
    sensorYMax = (sensor.x, sensor.y+sensor.distance)
    sensorYMin = (sensor.x, sensor.y-sensor.distance)

    result = set({})
    pointsUpperLeft = getPossiblePoints(sensor, sensorXMin, sensorYMin,  limit, -1)
    pointsUpperRight = getPossiblePoints(sensor, sensorYMin, sensorXMax, limit, 1)
    pointsLowerRight = getPossiblePoints(sensor, sensorXMax, sensorYMax, limit, 1)
    pointsLowerLeft = getPossiblePoints(sensor, sensorYMax, sensorXMin, limit, -1)

    pointsUpperLeft.union(pointsUpperRight).union(pointsLowerRight).union(pointsLowerLeft)
    result = result.union(pointsUpperLeft)
    result = result.union(pointsUpperRight)
    result = result.union(pointsLowerLeft)
    result = result.union(pointsLowerRight)
    if sensorYMin[1]-1>=0 and sensorYMin[1]-1<=limit:
        result.add((sensor.x, sensorYMin[1]-1))
    if sensorYMax[1]+1>=0 and sensorYMax[1]+1<=limit:
        result.add((sensor.x, sensorYMax[1]+1))
    return result

SENSOR CHECK1:  5 5 2


{(2, 5),
 (3, 4),
 (3, 6),
 (4, 3),
 (4, 7),
 (5, 2),
 (5, 8),
 (6, 3),
 (6, 7),
 (7, 4),
 (7, 6),
 (8, 5)}

In [50]:
xyMax = 20
xyMax = 4000000
points = list(map(lambda sensor: getPointsNextBorderOfSensor(sensor, xyMax), sensors))

sensors.sort(key = lambda sensor: sensor.distance, reverse=True)

for sensor in sensors:
    print("SENSOR CHECK2: ", sensor.x, sensor.y, sensor.distance)
    for i in range(0, len(points)):
        pointSet = points[i]
        print("Check Set ", str(i), len(pointSet))
        pointsToRemove = set()
        for point in pointSet:
            if getManhattanDistance((sensor.x, sensor.y), point) <= sensor.distance:
                pointsToRemove.add(point)
        for point in pointsToRemove:
            points[i].remove(point)
points = reduce(lambda x,y: x.union(y), points)


SENSOR CHECK1:  346753 3974683 584956
SENSOR CHECK1:  3718952 2421864 666458
SENSOR CHECK1:  919295 2535231 1427038
SENSOR CHECK1:  3982082 2935124 110930
SENSOR CHECK1:  3693260 3095908 32448
SENSOR CHECK1:  3273954 2072378 73857
SENSOR CHECK1:  2936799 2402146 664161
SENSOR CHECK1:  3153125 3571786 1001743
SENSOR CHECK1:  3892050 3718400 803176
SENSOR CHECK1:  1670487 618098 1962273
SENSOR CHECK1:  277848 3523156 296970
SENSOR CHECK1:  3412305 198361 1938511
SENSOR CHECK1:  2324630 2084344 664964
SENSOR CHECK1:  3718763 3088136 50179
SENSOR CHECK1:  303245 10539 962256
SENSOR CHECK1:  3230426 1961497 83510
SENSOR CHECK1:  3616662 3171194 137614
SENSOR CHECK1:  3673367 3084002 6283
SENSOR CHECK1:  3350734 2116841 192142
SENSOR CHECK1:  1348476 3646233 896735
SENSOR CHECK1:  2817552 1654754 803127
SENSOR CHECK1:  22462 1187401 883551
SENSOR CHECK1:  3982255 1687776 1019046
SENSOR CHECK1:  3995256 2821993 237235
SENSOR CHECK1:  501107 3777047 232966
SENSOR CHECK1:  2507803 2761101 19496

{(3316868, 2686239)}

In [52]:
for point in points:
    a = point[0]*xyMax+point[1]
    print(a)

13267474686239
